<a href="https://colab.research.google.com/github/antonum/Redis-Workshops/blob/main/02-Vector_Similarity_Search/02-RedisVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Search with RedisVL



[Always-on demo](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook generates vector embeddings using pre-trained `sentence-transformers/all-MiniLM-L6-v2` model from HuggingFace, loads them to Redis and runs Vector Similarity search against Redis database.

In [1]:
#install Redis client and Hugging Face sentence transformers
!pip install -q sentence_transformers git+https://github.com/RedisVentures/redisvl.git@readme-enhancement

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, w

Install Redis Stack locally

In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Connect to the Redis server

In [3]:
import os
import redis
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-12110.c82.us-east-1-2.ec2.cloud.redislabs.com"
#REDIS_PORT=12110
#REDIS_PASSWORD="pobhBJP7Psicp2gV0iqa2ZOc1WdXXXXX"

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"


In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from redisvl.utils.vectorize import HFTextVectorizer


# create a vectorizer
# choose your model from the huggingface website
hf = HFTextVectorizer(model="sentence-transformers/all-MiniLM-L6-v2")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# embed a sentence
test = hf.embed("This is a test sentence.")
#test[:10]

In [6]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

"""from sentence_transformers import SentenceTransformer

from redis import Redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
"""

tqdm.pandas()



### Embedding generation model

Here we are using `sentence-transformers/all-MiniLM-L6-v2` from HuggingFace. https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2



In [7]:
#from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#test=model.encode("This is a test sentence.")
#test[:]

Download 12k+ tweets

In [8]:
!wget https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv

--2024-02-05 22:14:05--  https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2486081 (2.4M) [text/plain]
Saving to: ‘Labelled_Tweets.csv’

Labelled_Tweets.csv 100%[===================>]   2.37M  --.-KB/s    in 0.02s   

2024-02-05 22:14:05 (98.1 MB/s) - ‘Labelled_Tweets.csv’ saved [2486081/2486081]



In [9]:
df = pd.read_csv('Labelled_Tweets.csv').drop(columns=['created_at','score'])
#df=df.head(100) #trim dataframe to fit results into 30MB Redis database
df


,id,full_text
0,1,@KennyDegu very very little volume. With $10T ...
1,2,#ES_F achieved Target 2780 closing above 50% #...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...
3,4,@Issaquahfunds Hedged our $MSFT position into ...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...
...,...,...
12415,12587,RT @PeterLBrandt: $SPX $ES_F \r\nFollowing thi...
12416,12588,RT @vieiraUAE: Fearless Alex Vieira Calls Best...
12417,12589,$spy $spx $qqq $ndx #nyse going from poking th...
12418,12590,RT @DavidScottAdams: On watch tomorrow // Pt. ...


### Generate Embeddings

Generate vector embeddings within the dataframe. This step can take 2-3 minutes on GPU runtime for all 12k records.

In [10]:
def text_to_embedding(text):
  return np.array(hf.embed(text), dtype=np.float32).tobytes()

#generate vector embeddings
df["text_embedding"] = df["full_text"].progress_apply(text_to_embedding)
df.head()

  0%|          | 0/12420 [00:00<?, ?it/s]

,id,full_text,text_embedding
0,1,@KennyDegu very very little volume. With $10T ...,b'*\x92\x81\xbd\x14h\x8b\xbdy\xdf\xe4\xbc\xc0\...
1,2,#ES_F achieved Target 2780 closing above 50% #...,b'Q\x1b\x02\xbd\x14~/\xbd\xa6z\xb1\xbcb\x99\xd...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...,b'\x10\xaa\xa3\xbd]}\x10\xbd\xc3\xe8\xb9=%\x08...
3,4,@Issaquahfunds Hedged our $MSFT position into ...,b'\xc4\x7f\xd1\xbc\xbe\n`\xbd79 =\xe4\xc0\xef=...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...,b'\xc6\r\x1e\xbdK\\\xd4\xbcH/\xa1\xbc\xebq7=\x...


### Create Helper Functions

- Save dataframe to Redis HASH
- Create RediSearch Index

In [11]:
tmp="""
def load_dataframe(redis, df, key_prefix="tweet", id_column="id", pipe_size=100):
    records = df.to_dict(orient="records")
    pipe = redis.pipeline(transaction=False)
    i=1
    for record in tqdm(records):
        i=i+1
        key = f"{key_prefix}:{record[id_column]}"
        pipe.hset(key, mapping=record)
        if (i+1) % pipe_size == 0:
          res=pipe.execute()
    pipe.execute()

def create_redis_index(redis, idxname="tweet:idx"):
  try:
    redis.ft(idxname).dropindex()
  except:
    print("no index found")

  # Create an index
  indexDefinition = IndexDefinition(
      prefix=["tweet:"],
      index_type=IndexType.HASH,
  )

  redis.ft(idxname).create_index(
      (
          TextField("full_text", no_stem=False, sortable=False),
          VectorField("text_embedding", "HNSW", {  "TYPE": "FLOAT32",
                                                    "DIM": 384,
                                                    "DISTANCE_METRIC": "COSINE",
                                                  })
      ),
      definition=indexDefinition
  )

"""

In [12]:
schema = {
    "index": {
        "name": "tweet:idx",
        "prefix": "tweet:",
        "storage_type": "hash", # default setting -- HASH
    },
    "fields": [
        {"name": "full_text", "type": "text"},
        {
            "name": "text_embedding",
            "type": "vector",
            "attrs": {
                "dims": 384,
                "distance_metric": "cosine",
                "algorithm": "HNSW",
                "datatype": "float32"
            }

        }
    ],
}

### Create index and load data to Redis

In [13]:
# clear Redis database (optional)
#redis.flushdb()

from redisvl.index import SearchIndex

index = SearchIndex.from_dict(schema)

# create Index
#create_redis_index(redis)

# load data from Dataframe to Redis HASH
#load_dataframe(redis,df,key_prefix="tweet", pipe_size=100)
index.connect("redis://localhost:6379")

index.create(overwrite=True)

#keys = index.load(df.to_dict())
#df.to_dict('records')
keys = index.load(df.to_dict('records'))
#keys

In [14]:
#Check how the data is stored in Redis
#!redis-cli $REDIS_CONN keys "*"
#!redis-cli $REDIS_CONN hgetall "tweet::c4b63c9494194d4ea65b7e265bcd2d5f"

## Query the database

[Alway-on Streamlit app](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)


Try queries like:
“Oil”, “Oil Reserves”, “Fossil fuels”

Lexical Full Text search quickly runs out of matches

Vector search continues to discover relevant tweets

In [15]:
user_query="oil price"
# queries to try "oil reserve", "fossil fuels"

In [16]:
from redisvl.query import FilterQuery
from redisvl.query.filter import Text

# exact match filter -- document must contain the exact word doctor
text_filter = Text("full_text") == user_query

filter_query = FilterQuery(
    return_fields=["full_text"],
    filter_expression=text_filter
)

results = index.query(filter_query)

pd.DataFrame(results)
#results

,id,full_text
0,tweet::c5a35b98b10b461bbb3060fe9bcc9e75,Today's book recommendation goes for the winne...
1,tweet::abd86259baba44e3bc4cf052f61e3f9b,RT @SeekingAlpha: $XOM - Despite The Oil Price...
2,tweet::13f93ebaa2e546d1b27f6df08a64281a,RT @SeekingAlpha: $XOM - Despite The Oil Price...
3,tweet::b4ca969d2de042deafc808dd65849eda,RT @SeekingAlpha: $XOM - Despite The Oil Price...
4,tweet::d9d3e0358709421c80d0219cd0e0dd45,RT @SeekingAlpha: $XOM - Despite The Oil Price...
5,tweet::c86bf6cf6a9f4b15b6d0456d865711a0,RT @CharlesSizemore: Investors believe that Sa...
6,tweet::3e77789003a14f039adc89ef3177bf74,RT @SeekingAlpha: $EURN - Euronav's CEO On Sur...
7,tweet::51a7c77ae7a04bfea0cf7e7312edadaa,$EURN - Euronav's CEO On Surging Rates Amid Th...
8,tweet::e266bf30fcb148e1abd457289c5ad1da,Investors believe that Saudi Arabia is winning...
9,tweet::cefe64f8f7f24ecdaa73fe596fe57c53,Despite The Oil Price Carnage Exxon Will Grow ...


In [17]:
#using Full Text Index
#q = Query(user_query)\
#  .return_fields("full_text")
#res = redis.ft("tweet:idx").search(q)
#if res.total==0:
#  print("No matches found")
#else:
#  res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
#  display(res_df)

In [18]:
from redisvl.query import VectorQuery
#from jupyterutils import result_print

query = VectorQuery(
    vector=hf.embed(user_query),
    vector_field_name="text_embedding",
    return_fields=["full_text", "vector_distance"],
    num_results=10
)
results = index.query(query)
pd.DataFrame(results)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,full_text
0,tweet::1c50fbdcb8364c299bb5ce72fc20f8db,0.369450867176,Would you spend $2 more a gallon of gasoline i...
1,tweet::6cb920246bd44dcb8b625bd298afd218,0.371095597744,RT @tradingcrudeoil: Crude oil closed up $0.48...
2,tweet::7f035669e53b4428b1123b07560c7599,0.381934225559,..and oil still 25.74 LMAO &gt;&gt;&gt;NO DEM...
3,tweet::82a757c0c1ac4ed2b4b8dc01579dfc90,0.396132230759,Bad news for #oil. It’s going to between $10 ...
4,tweet::2a6c4847564043d4b9bf448f47733d8e,0.409308671951,Oil erases gains for the day in fall to $25 ht...
5,tweet::9c56846282ad4065a4026150a9eb3501,0.429816782475,Do higher oil prices help the consumer and sma...
6,tweet::8e26b89fbed44579bb3624e3a2724d2b,0.430081188679,The price of Texas intermediate oil (WTI) slum...
7,tweet::10736b4ae9fd46db94b599d613b34a49,0.431391596794,#OIL Sentiment ($22.50)\r\n\r\nWhat’s next for...
8,tweet::8961b69885a34aed9c60a8274813748b,0.441844820976,OH how bullish for #oil LOL\r\n\r\n#OOTT #Oi...
9,tweet::8f458880b8bc45cf8af3d505541c40b1,0.442979991436,$DXY 99.55-0.57%&lt;==US Dollar lower #Fed $2....


In [19]:
#using Vector Similarity Index
#query_vector=text_to_embedding(user_query)
#q = Query("*=>[KNN 10 @text_embedding $vector AS result_score]")\
#                .return_fields("result_score","full_text")\
#                .dialect(2)\
#                .sort_by("result_score", True)
#res = redis.ft("tweet:idx").search(q, query_params={"vector": query_vector})
#print(res)
#res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
#res_df